In [3]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import csv
import tensorflow as tf
from tensorflow.keras.models import Model
from neurite.tf import models  # Assuming the module's location
import voxelmorph.tf.losses as vtml
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Input, BatchNormalization
from tensorflow.keras.layers import Lambda
import neurite as ne
import sys
import nibabel as nib
from tensorflow.keras.models import load_model
from neurite_sandbox.tf.models import labels_to_labels
import nibabel as nib
import tqdm
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import neurite as ne
import voxelmorph as vxm
from utils import *
import pathlib
import surfa as sf
import numpy as np

import os
import numpy as np
import surfa as sf
from utils import resize
import re
import json
import numpy as np
from sklearn.metrics import f1_score

def process_image(im, unet_model):
    a = im.copy()
    return unet_model.predict(a[None,...,None], verbose=0)
    
def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred)
    return (2.0 * intersection) / (union) 

def dynamic_resize(image, target_width=192):   

    fov = np.multiply(image.shape, image.geom.voxsize)

    new_voxsize = fov / target_width

    new_voxsize = np.max(new_voxsize[:2])  # ignore slice thickness
    return new_voxsize
    
def my_hard_dice(y_true, y_pred):
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()
    dice = dice_coefficient(y_true_flat, y_pred_flat)
    return dice
from skimage.measure import label   

def getLargestCC(segmentation):
    labels = label(segmentation)
    assert( labels.max() != 0 ) # assume at least 1 CC
    largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1
    return largestCC
    
def soft_dice(a, b):
    dim = len(a.shape) - 2
    space = list(range(1, dim + 1))
    # print("dim",dim,"space",space,"a.dim",a.shape,"b.dim",b.shape)
    top = 2 * tf.reduce_sum(a * b, axis=space)
    bot = tf.reduce_sum(a ** 2, axis=space) + tf.reduce_sum(b ** 2, axis=space)
    
    out = tf.divide(top, bot + 1e-6)
    return -tf.reduce_mean(out)

def my_dice_coefficient(y_true, y_pred, smooth=1e-8):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred)
    return (2.0 * intersection + smooth) / (union + smooth)
    
def minmax_norm(x):
    x_min = np.min(x)
    x_max = np.max(x)
    return (x - x_min) / (x_max - x_min)

def find_manual_mask(filename):
    base_filename, _ = os.path.splitext(filename)  # Separate filename and extension
    pattern = re.compile(rf"manual_masks_b\d+_mom_\d+_{re.escape(base_filename)}_segment\.nii\.gz")

    matches = [f for f in os.listdir("mgh_2d") if pattern.match(f)]
    
    if matches:
        return matches[0]

def percentile_norm(x, percentile=90):
    p_value = np.percentile(x, percentile)
    
    x_clipped = np.clip(x, 0, p_value)
    
    x_min_clipped = np.min(x_clipped)
    x_max_clipped = np.max(x_clipped)
    normalized_x = (x_clipped - x_min_clipped) / (x_max_clipped - x_min_clipped)
    return normalized_x
    
tf.get_logger().setLevel('ERROR')

warp_blur_min=np.array([2, 4, 8])
bias_blur_min=np.array([2, 4, 8])
bias_blur_max=bias_blur_min*2





random.seed(3000)

def load_model3D(checkpoint_path,gmm, norm, num_dim=3,nb_labels=2,
                                            dimx=192,
                                            dimy=192,
                                            dimz=192,
                                            batch_size=8,
                                            warp_max=2.5,
                                            warp_min=.5,
                                            warp_blur_min=np.array([2, 4, 8]),
                                            warp_blur_max=warp_blur_min*2,
                                            bias_blur_min=np.array([2, 4, 8]),
                                            bias_blur_max=bias_blur_min*2,
                                            initial_lr=1e-4,
                                            lr = 1e-4,
                                            lr_lin = 1e-4,
                                            nb_levels=5,
                                            conv_size=3,
                                            num_epochs=40000,
                                            num_bg_labels=16,
                                            nb_conv_per_level=2):


    input_img = Input(shape=(dimx, dimy, dimz, 1))
    
    unet_model = vxm.networks.Unet(inshape=(dimx, dimy, dimz, 1), nb_features=(en, de), 
                                   nb_conv_per_level=nb_conv_per_level,
                                   final_activation_function='softmax')
    if gmm:

        # input_img = Input(shape=(*in_shape,1))
        input_brain = get_brain_tf(input_img)
        _, y_brain = model1(input_brain)

        input_fov = get_fov_tf(input_img)
        
        _, y_fov = model2(input_fov)
        # input_final = y_brain + y_fov * (y_brain == 0)
        input_final = y_brain + tf.cast(y_fov, dtype=tf.int32) * tf.cast(y_brain == 0, dtype=tf.int32)

        generated_img, y = labels_to_image_model(input_final)
        segmentation = unet_model(generated_img)
        combined_model = Model(inputs=input_img, outputs=segmentation)
    elif norm:
        x = BatchNormalization(axis=-1)(input_img)  # Change the axis according to your needs
        segmentation = unet_model(x)
        combined_model = Model(inputs=input_img, outputs=segmentation)
    else:
        segmentation = unet_model(input_img)
        combined_model = Model(inputs=input_img, outputs=segmentation)

    if os.path.exists(checkpoint_path):
        combined_model.load_weights(checkpoint_path)
        print("Loaded weights from the checkpoint and continued training.")
    else:
        # print(checkpoint_path)
        print("Checkpoint file not found.")
    unet_model = combined_model.layers[-1]

    return unet_model


num_row = 3
per_row = 10

    
def calculate_dice_coefficient(ground_truth, predicted):
    intersection = np.sum(np.logical_and(ground_truth, predicted))
    union = np.sum(np.logical_or(ground_truth, predicted))
    dice_coefficient = (2.0 * intersection) / (union + intersection)
    return dice_coefficient

def print_row_dice_coefficients(i, row_dice_coefficients):
    formatted_dice = [f"   {dice:.4f}" for j, dice in enumerate(row_dice_coefficients)]
    num_spaces = num_row-1
    padding_left = 0
    padding_right = 0

    formatted_output = f"{' ' * padding_left}{'       '.join(formatted_dice)}{' ' * padding_right}"
    print(f' {formatted_output}')
    
def visualize(combined_model,image,mask, id):
    crop_img = image.resize([1.5, 1.5, 1.5]).reshape((dim_, dim_, dim_,1))    
    crop_data = minmax_norm(crop_img.data)
    
    prediction_one_hot = process_image(crop_data, combined_model)
    predictions_argmax = np.argmax(prediction_one_hot, axis=-1)
    prediction = np.squeeze(predictions_argmax, axis=0)

    prediction = sf.Volume(prediction)#.reshape(mask.data.shape)#.resample_like(mask)
    # mask.data[mask.data != 0] = 1
    hard_dice_score = my_hard_dice(mask.data, prediction.data)
    
    print(f"Hard Dice Coefficient: {hard_dice_score}")

    nib.save(nib.Nifti1Image(crop_img.astype(np.float32), np.eye(4), header=None), f"output/crop_img_{id}.nii.gz")
    nib.save(nib.Nifti1Image(prediction.data.astype(np.int32), np.eye(4), header=None), f"output/pred_{id}.nii.gz")
    nib.save(nib.Nifti1Image(mask.data.astype(np.int32), np.eye(4), header=None), f"output/mask_{id}.nii.gz")

    ne.plot.volume3D(crop_img);
    ne.plot.volume3D(mask);
    ne.plot.volume3D(prediction);

from scipy.ndimage import binary_closing
from scipy import ndimage

def calculate_hard_dice(image, model, mask):
    # mask = sf.load_volume(mask_filename).reshape((dim_, dim_, dim_))


    prediction_one_hot = process_image(image.data, model)
    predictions_argmax = np.argmax(prediction_one_hot, axis=-1)
    prediction = np.squeeze(predictions_argmax, axis=0)
    mask.data[mask.data != 0] = 1
    # kernel = np.ones((5, 5, 5), np.uint8)  # Adjust the kernel size as needed for 3D
    # for i in range(5):
    dial_num = 4
    prediction = binary_dilation(prediction, iterations=dial_num)
    prediction = ndimage.binary_fill_holes(prediction).astype(int)
    prediction = binary_erosion(prediction, iterations=dial_num)
    prediction = getLargestCC(prediction)

    # prediction = ndimage.binary_fill_holes(prediction).astype(int)
    # prediction = getLargestCC(prediction)
    # prediction = binary_dilation(prediction, iterations=10)
    # prediction = binary_erosion(prediction, iterations=10)
    
    return my_hard_dice(prediction.flatten(),mask.data.flatten())

def evaluate(validation_folder_path, combined_model , new_voxsize):
    subfolders = [f.name for f in os.scandir(validation_folder_path) if f.is_dir()]


    
    b_hard_dice_scores=[]

    b_salehi_dice_scores=[]
    b_kevin_dice_scores=[]
    b_ebner_dice_scores=[]

    latest_images=[]
    latest_masks=[]
    
    end_epoch = 0 + step_size * ((initial_epoch) // step_size)
    

    
    b_10_hard_dice_scores = []
    b_10_salehi_dice_scores = []
    b_10_ebner_dice_scores = []
    b_10_kevin_dice_scores = []

    for folder in subfolders:
        # print(folder)
        folder_path = os.path.join(validation_folder_path, folder)
        # filename = os.path.join(folder_path,"image.mgz")
        if os.path.exists(os.path.join(folder_path, "image.mgz")): 
            filename = os.path.join(folder_path, "image.mgz") 
        else:
            filename = os.path.join(folder_path, "image.nii.gz")
# os.path.join(folder_path, "image.nii.gz") if os.path.exists(os.path.join(folder_path, "image.nii.gz"))

        mask_filename = os.path.join(folder_path,"manual.nii.gz")
        ebner_mask_filename = os.path.join(folder_path,"ebner.nii.gz")
        kevin_mask_filename = os.path.join(folder_path,"keraudren.nii.gz")
        salehi_mask_filename = os.path.join(folder_path,"salehi.nii.gz")

        
        image = sf.load_volume(filename)
        orig_voxsize = image.geom.voxsize
        if validation_folder_path == "EPI_150":
            new_voxsize = [dynamic_resize(image)]*3
        else:
            new_voxsize = [dynamic_resize(image)]*3

        crop_img = image.resize([orig_voxsize[0],orig_voxsize[1],new_voxsize[-1]], method="linear")
        crop_img = crop_img.resize(new_voxsize, method="linear").reshape([dim_, dim_, dim_])
        # crop_data = minmax_norm(crop_img.data)
        crop_data = percentile_norm(crop_img.data)
        # crop_data = minmax_norm(crop_data)
    
        mask = sf.load_volume(mask_filename).resize([orig_voxsize[0],orig_voxsize[1],new_voxsize[-1]], method="linear")
        mask = mask.resize(new_voxsize).reshape([dim_, dim_, dim_, 1])

        salehi_mask = sf.load_volume(salehi_mask_filename).resize([orig_voxsize[0],orig_voxsize[1],new_voxsize[-1]], method="linear")
        ebner_mask = sf.load_volume(ebner_mask_filename).resize([orig_voxsize[0],orig_voxsize[1],new_voxsize[-1]], method="linear")
        kevin_mask = sf.load_volume(kevin_mask_filename).resize([orig_voxsize[0],orig_voxsize[1],new_voxsize[-1]], method="linear")

        salehi_mask = salehi_mask.resize(new_voxsize).reshape([dim_, dim_, dim_, new_voxsize[-1]])
        ebner_mask = ebner_mask.resize(new_voxsize).reshape([dim_, dim_, dim_, 1])
        kevin_mask = kevin_mask.resize(new_voxsize).reshape([dim_, dim_, dim_, 1])

        mask.data[mask.data != 0] = 1
        hard_dice_score = calculate_hard_dice(crop_img, combined_model, mask)
        # mom = int(folder.split("_")[1])
        
        salehi_dice_score = my_hard_dice(salehi_mask.data, mask.data)
        ebner_dice_score = my_hard_dice(ebner_mask.data, mask.data)
        kevin_dice_score = my_hard_dice(kevin_mask.data, mask.data)

        print("file: {:s}, Dice: {:.4f}, Kevin: {:.4f}, Ebner: {:.4f}, Salehi: {:.4f}".format(
            folder, hard_dice_score, kevin_dice_score, ebner_dice_score, salehi_dice_score
        ))
        # print(folder, hard_dice_score)
        b_10_hard_dice_scores.append(hard_dice_score)

        b_10_salehi_dice_scores.append(salehi_dice_score)
        b_10_ebner_dice_scores.append(ebner_dice_score)
        b_10_kevin_dice_scores.append(kevin_dice_score)

    b_hard_dice_scores= (np.mean(b_10_hard_dice_scores),np.std(b_10_hard_dice_scores))
    
    b_salehi_dice_scores=(np.mean(b_10_salehi_dice_scores),np.std(b_10_salehi_dice_scores))
    b_kevin_dice_scores = (np.mean(b_10_kevin_dice_scores),np.std(b_10_kevin_dice_scores))
    b_ebner_dice_scores= (np.mean(b_10_ebner_dice_scores),np.std(b_10_ebner_dice_scores))

    return b_hard_dice_scores , b_salehi_dice_scores, b_kevin_dice_scores, b_ebner_dice_scores
        
def load_models():
    with open("params_192.json", "r") as json_file:
        config = json.load(json_file)

    model1_config = config["brain"]
    model2_config = config["body"]
    model3_config = config["labels_to_image_model"]
    model4_config = config["labels_to_image_model_with_shapes"]
    
    # Convert labels_out keys to integers for all models
    model1_config["labels_out"] = {int(key): value for key, value in model1_config["labels_out"].items()}
    model2_config["labels_out"] = {int(key): value for key, value in model2_config["labels_out"].items()}
    model3_config["labels_out"] = {int(key): value for key, value in model3_config["labels_out"].items()}
    
    model4_config["labels_out"] = {int(key): value for key, value in model3_config["labels_out"].items()}
    # Now you have the modified configuration
    # Brain
    model1 = create_model(model1_config)
    # Body
    model2 = create_model(model2_config)
    # Model
    labels_to_image_model = create_model(model3_config)
    return model1, model2, labels_to_image_model
model1, model2, labels_to_image_model = load_models()
import os, shutil, glob


gmm = False
synth=True
conc=True
norm=True
if gmm:
    step_size=1000
    # models_dir='models_synth_gmm_mgh_brain_mgh_body_192'
    if synth:
        # models_dir='models_synth_gmm_feta_mgh_body_192'
        models_dir='models_synth_gmm_mgh_brain_mgh_body_192'
elif conc:
    step_size=3000
    if synth and norm:
        models_dir='models_synth_less_conc_norm_shapes_feta_192'
    elif synth and not norm:
        models_dir='models_synth_less_conc_shapes_feta_192'
    else:
        models_dir='models_less_conc_shapes_feta_192'
else:
    # models_dir='models_shapes_feta_192'
    if synth:
        step_size=100
        models_dir='models_synth_shapes_feta_192'
    else:
        step_size=500
        models_dir='models_shapes_feta_192'


if synth:
    en = [16, 16, 32 ,32 ,64 ,64 ,64 ,64 ,64 ,64 ,64 ,64 ,64]
    de = [64 ,64 ,64 ,64, 64 ,64 ,64, 64, 64, 32 ,32 ,16 ,16 ,2]
else:
    en=[16,16, 32,32, 64,64,128,128, 192,192]
    de=[192,192, 128,128,64,64,32,32,16,16,2]


nb_features = '_'.join(map(str, en))
dim_=192




last_models_hard_dice_scores = []
n = 500
# for i in range(latest_epoch, latest_epoch - n, -10):
latest_weight = max(glob.glob(os.path.join(models_dir, 'weights_epoch_*.h5')), key=os.path.getctime, default=None)

match = re.search(r'(\d+)', latest_weight.split("/")[1])
initial_epoch = int(match.group())

checkpoint_path=models_dir+'/weights_epoch_'+str(initial_epoch)+'.h5'
print(checkpoint_path)
combined_model = load_model3D(checkpoint_path,gmm,norm , num_dim=3, dimx=dim_,
                                                       dimy=dim_,
                                                       dimz=dim_)

# new_voxsize = [1.25,1.25,1.25] # for MGH
new_voxsize = [0.65,0.65,0.65] # For EPI
# validation_folder_path_1="test/haste_trim2"
# validation_folder_path_2="test/haste_trim3"

validation_folder_path_1="validation2/haste_trim2"
validation_folder_path_2="validation2/haste_trim3"

validation_folder_path_3="EPI_150"

# b2_fstrip, b2_salehi, b2_kevin , b2_ebner= evaluate(validation_folder_path_1,combined_model, new_voxsize)
# b3_fstrip, b3_salehi, b3_kevin , b3_ebner = evaluate(validation_folder_path_2,combined_model,new_voxsize)

epi_fstrip, epi_salehi, epi_kevin , epi_ebner = evaluate(validation_folder_path_3,combined_model,new_voxsize)


import matplotlib.pyplot as plt
import numpy as np

import matplotlib.pyplot as plt

# Example data with one mean and one std for each baseline
second_trimester_data = {
    'Salehi': b2_salehi,
    'Keraudren': b2_kevin,
    'Ebner': b2_ebner,
    'f-Strip': b2_fstrip
}

third_trimester_data = {
    'Salehi': b3_salehi,
    'Keraudren': b3_kevin,
    'Ebner': b3_ebner,
    'f-Strip': b3_fstrip
}

epi_data = {
    'Salehi': epi_salehi,
    'Keraudren': epi_kevin,
    'Ebner': epi_ebner,
    'f-Strip': epi_fstrip
}

# Create separate plots for each trimester
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 6))

# Rectangular box plot for the second trimester
bplot1 = axes[0].boxplot(list(second_trimester_data.values()),
                          vert=True,
                          patch_artist=True,
                          labels=list(second_trimester_data.keys()))  # will be used to label x-ticks

# Set different colors for boxes
colors1 = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow']
for patch, color in zip(bplot1['boxes'], colors1):
    patch.set_facecolor(color)

axes[0].set_title('Second Trimester', fontsize=18)
axes[0].set_ylabel('Dice Score', fontsize=15)
axes[0].tick_params(axis='both', labelsize=15)

# Rectangular box plot for the third trimester
bplot2 = axes[1].boxplot(list(third_trimester_data.values()),
                          vert=True,
                          patch_artist=True,
                          labels=list(third_trimester_data.keys()))  # will be used to label x-ticks

# Set different colors for boxes
colors2 = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow']
for patch, color in zip(bplot2['boxes'], colors2):
    patch.set_facecolor(color)

axes[1].set_title('Third Trimester', fontsize=18)
axes[1].set_ylabel('Dice Score', fontsize=15)
axes[1].tick_params(axis='both', labelsize=15)

# Rectangular box plot for epi_data
bplot3 = axes[2].boxplot(list(epi_data.values()),
                          vert=True,
                          patch_artist=True,
                          labels=list(epi_data.keys()))  # will be used to label x-ticks

# Set different colors for boxes
colors3 = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow']
for patch, color in zip(bplot3['boxes'], colors3):
    patch.set_facecolor(color)

axes[2].set_title('Epi Data', fontsize=18)
axes[2].set_ylabel('Dice Score', fontsize=15)
axes[2].tick_params(axis='both', labelsize=15)

plt.tight_layout()
plt.show()


(None, 192, 192, 192, 1)
(None, 192, 192, 192, 1)
(None, 192, 192, 192, 1)
(None, 192, 192, 192, 1)
(None, 192, 192, 192, 1)
(None, 192, 192, 192, 1)
models_synth_less_conc_norm_shapes_feta_192/weights_epoch_300.h5
Loaded weights from the checkpoint and continued training.
file: epiSurvey_25w1d_046_f07, Dice: 0.1421, Kevin: 0.0020, Ebner: 0.8013, Salehi: 0.0002
file: epiSurvey_35w3d_025_f12, Dice: 0.0206, Kevin: 0.0000, Ebner: 0.4859, Salehi: 0.0594
file: T2sEPI_012_26w5d_041_f19, Dice: 0.0833, Kevin: 0.0025, Ebner: 0.8765, Salehi: 0.0000
file: epiSurvey_35w3d_026_f09, Dice: 0.0016, Kevin: 0.0000, Ebner: 0.1488, Salehi: 0.0004
file: epiSurvey_23w1d_029_f09, Dice: 0.0332, Kevin: 0.0016, Ebner: 0.7256, Salehi: 0.0020
file: epiSurvey_29w4d_044_f11, Dice: 0.2041, Kevin: 0.0041, Ebner: 0.0000, Salehi: 0.0045
file: epiSurvey_35w4d_032_f05, Dice: 0.0273, Kevin: 0.0000, Ebner: 0.0000, Salehi: 0.0000
file: epiSurvey_36w2d_023_f03, Dice: 0.1352, Kevin: 0.0001, Ebner: 0.5378, Salehi: 0.0023
file:

NameError: name 'b2_salehi' is not defined

In [38]:
# image = sf.load_volume("test/haste_trim3/mom_011__30wk__8T2_Haste_Sag_mm_400_FOV/image.mgz")
# mask = np.zeros(image.shape)
# nib.save(nib.Nifti1Image(mask.astype(np.int32), np.eye(4), header=None), f"test/haste_trim3/mom_011__30wk__8T2_Haste_Sag_mm_400_FOV/salehi.nii.gz")
# nib.save(nib.Nifti1Image(mask.astype(np.int32), np.eye(4), header=None), f"test/haste_trim3/mom_011__30wk__8T2_Haste_Sag_mm_400_FOV/ebner.nii.gz")
# nib.save(nib.Nifti1Image(mask.astype(np.int32), np.eye(4), header=None), f"test/haste_trim3/mom_011__30wk__8T2_Haste_Sag_mm_400_FOV/keraudren.nii.gz")


In [24]:
b2_fstrip

(0.3695936228104438, 0.259286525217361)

In [34]:
# subfolders = [f.name for f in os.scandir("EPI_256/labels") if f.is_dir()]


In [35]:
# for folder in subfolders:
#     folder_path = os.path.join("EPI", folder)
#     filename = os.path.join(folder_path,"image.nii.gz")
#     image = sf.load_volume(filename)
#     vsize = image.geom.voxsize
#     i_shape = image.shape
#     image= image.reshape([256,256,i_shape[2]])
#     # sf.save()
#     nib.save(nib.Nifti1Image(image, np.eye(4)), f"EPI_256/"+folder+".nii.gz")

#     .reshape([256,256, ])
    
# image = sf.load_volume("EPI/T2sEPI_007_21w4d_041_f04/image.nii.gz")


In [94]:
image.shape
# /autofs/space/bal_004/users/jd1677/kevin-2016/data/b
# image = sf.load_volume("/autofs/space/bal_004/users/jd1677/synthstrip/EPI_256/T2sEPI_020_21w1d_035_f15/image.nii.gz")

In [95]:
# nib.save(nib.Nifti1Image(np.zeros(image.shape), np.eye(4)), "/autofs/space/bal_004/users/jd1677/synthstrip/EPI_256/T2sEPI_020_21w1d_035_f15/salehi.nii.gz")

In [53]:
# a= np.zeros(image.shape)

In [54]:
# a.shape

(256, 256, 128)

In [11]:
# from os import listdir
# from os.path import isfile, join
# onlyfiles = [f for f in listdir("EPI_256/labels") if isfile(join("EPI_256/labels", f))]
# for file in onlyfiles:
#     folder_path = os.path.join("EPI_256/labels", file)
#     # filename = os.path.join(folder_path,"image.nii.gz")
#     image = sf.load_volume(folder_path)
#     # vsize = image.geom.voxsize
#     i_shape = image.shape
#     image= image.reshape([256,256,i_shape[2]])
#     # sf.save()
#     print(image.shape)
#     nib.save(nib.Nifti1Image(image, np.eye(4)), f"EPI_256/labels/"+file)

#     # .reshape([256,256, ])
    
# # image = sf.load_volume("EPI/T2sEPI_007_21w4d_041_f04/image.nii.gz")

(256, 256, 70)
(256, 256, 100)
(256, 256, 70)
(256, 256, 70)
(256, 256, 70)
(256, 256, 96)
(256, 256, 90)
(256, 256, 70)
(256, 256, 90)
(256, 256, 70)
(256, 256, 70)
(256, 256, 94)
(256, 256, 57)
(256, 256, 90)
(256, 256, 70)
(256, 256, 88)
(256, 256, 70)
(256, 256, 40)
(256, 256, 70)
(256, 256, 66)
(256, 256, 70)
(256, 256, 99)
(256, 256, 80)
(256, 256, 58)
(256, 256, 70)
(256, 256, 70)
(256, 256, 46)
(256, 256, 78)
(256, 256, 70)
(256, 256, 70)
(256, 256, 80)
(256, 256, 70)
(256, 256, 70)
(256, 256, 74)
(256, 256, 128)
(256, 256, 70)
(256, 256, 70)
(256, 256, 85)
(256, 256, 40)
(256, 256, 80)
(256, 256, 66)
(256, 256, 70)
(256, 256, 128)
(256, 256, 90)
(256, 256, 80)
(256, 256, 74)
(256, 256, 48)
(256, 256, 70)
(256, 256, 52)
(256, 256, 70)
(256, 256, 70)
(256, 256, 70)
(256, 256, 80)
(256, 256, 85)
(256, 256, 90)
(256, 256, 75)
(256, 256, 58)
(256, 256, 63)
(256, 256, 37)
(256, 256, 70)
(256, 256, 70)
(256, 256, 40)
(256, 256, 70)
(256, 256, 70)
(256, 256, 70)
(256, 256, 70)
(256, 2